<img src="img/headerImg.jpeg" alt="Drawing" width="1000" height="100"/>

# Create Your First Graph

TigerGraph is a graph database software with a multitude of functionality and solutions to some of the issues that have plagued other graph databases. This notebook demonstrates how to use basic commands to connect, create and load data into TigerGraph using the Python pyTigerGraph module.

## STEP 1: Import Packages

Note: Assuming you have installed the pyTigerGraph package. If not install it using:
```pip install pyTigerGraph```

In [1]:
import pyTigerGraph as tg
import pandas as pd
import json

## STEP 2: Establishing the connection to a TigerGraph database

<div>
  <img style="vertical-align:top" src="img/connected-icon.png" width="30" height="30"/>
  <span style="">The functionality of pyTigerGraph is implemented by the TigerGraphConnection class. To establish the connection to the database you need to provide the hostname, username and password to access the database.</span>
</div>

In [2]:
conn = tg.TigerGraphConnection(
    host='http://localhost',
    username="tigergraph",
    password='tigergraph')

## STEP 3: Define a Graph Schema

<div>
  <img style="vertical-align:top" src="img/graph_img.png" width="30" height="30"/>
  <span style="">Before data can be loaded into the graph store, the user must define a graph schema. A graph schema is a "dictionary" that defines the types of entities, vertices and edges, in the graph and how those types of entities are related to one another.</span>
</div>

### WARNING: DROP ALL - Will Delete everything in your graph!

Execute this cell if you would like to start the notebook lab from the beginning.

In [3]:
print(conn.gsql('''DROP ALL''', options=[]))

Dropping all, about 1 minute ...
Abort all active loading jobs
Try to abort all loading jobs on graph social, it may take a while ...
[ABORT_SUCCESS] No active Loading Job to abort.
Resetting GPE...
Successfully reset GPE
Stopping GPE GSE
Successfully stopped GPE GSE in 7.356 seconds
Clearing graph store...
Successfully cleared graph store
Starting GPE GSE RESTPP
Successfully started GPE GSE RESTPP in 0.062 seconds
Everything is dropped.


----
The CREATE VERTEX statement defines a new global vertex type, with a name and an attribute list. 

The CREATE EDGE statement defines a new global edge type. There are two forms of the CREATE EDGE statement, one for directed edges and one for undirected edges.  Each edge type must specify that it connects FROM one vertex type TO another vertex type.

In [4]:
print(conn.gsql('''
CREATE VERTEX person (PRIMARY_ID name STRING, gender STRING, name STRING, age INT, state STRING)                     
CREATE UNDIRECTED EDGE friendship (FROM person, TO person, connect_day datetime)
CREATE GRAPH social (person, friendship)
''', options=[]))

The vertex type person is created.
The edge type friendship is created.
Stopping GPE GSE RESTPP
Successfully stopped GPE GSE RESTPP in 0.310 seconds
Starting GPE GSE RESTPP
Successfully started GPE GSE RESTPP in 0.065 seconds
The graph social is created.


----
The GSQL command enable sending arbitrary GSQL statements to the database. Next cell show how to test the schema createtion was succesful.

In [5]:
print(conn.gsql('''ls''', options=[]))

---- Global vertices, edges, and all graphs
Vertex Types:
- VERTEX person(PRIMARY_ID name STRING, gender STRING, name STRING, age INT, state STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE"
Edge Types:
- UNDIRECTED EDGE friendship(FROM person, TO person, connect_day DATETIME)

Graphs:
- Graph social(person:v, friendship:e)
Jobs:


JSON API version: v2
Syntax version: v1



## Step 3: Load data

<div>
  <img style="vertical-align:top" src="img/load_data.png" width="30" height="30"/>
  <span style="">The pyTigerGraph submodule provides results from various built-in endpoints in a Pandas DataFrame. To load data upload the csv file to a dataframe inside the notebook. 
</span>
</div>

In [6]:
persons = pd.read_csv('data/persons.csv')
persons

,name,gender,age,state
0,Tom,male,40,ca
1,Dan,male,34,ny
2,Jenny,female,25,tx
3,Kevin,male,28,az
4,Amily,female,22,ca
5,Nancy,female,20,ky
6,Jack,male,26,fl


In [7]:
friendships = pd.read_csv('./data/friendships.csv')
friendships

,person1,person2,date
0,Tom,Dan,2017-06-03
1,Tom,Jenny,2015-01-01
2,Dan,Jenny,2016-08-03
3,Jenny,Amily,2015-06-08
4,Dan,Nancy,2016-01-03
5,Nancy,Jack,2017-03-02
6,Dan,Kevin,2015-12-30


----
Change to the social graph

In [8]:
socialGraph=tg.TigerGraphConnection(host='http://localhost', graphname='social' )
results = socialGraph.getSchema()

In [9]:
v_person = socialGraph.upsertVertexDataFrame(persons, "person", "name", attributes={"name": "name", "gender": "gender", "age": "age", "state": "state"})
print(str(v_person) + " Customer VERTICES Upserted")

7 Customer VERTICES Upserted


In [10]:
numPersons = socialGraph.getVertexCount("person")
print(f"There are currently {numPersons} in of vertex type person, prior to map")

There are currently 0 in of vertex type person, prior to map


In [11]:
v_friendships = socialGraph.upsertEdgeDataFrame(friendships,"person", "friendship", "person", from_id="person1", to_id="person2", attributes={"connect_day":"date"})
print(str(v_friendships) + " Friendships Edges Upserted")

7 Friendships Edges Upserted


## Step 4: Explore Graph

<div>
  <img style="vertical-align:top" src="img/inquiry.jpeg" width="28" height="28"/>
  <span style="">TBD. 
</span>
</div>

In [12]:
socialGraph.getVertexStats('person')

{'person': {'age': {'MAX': 40, 'MIN': 20, 'AVG': 27.85714}}}

In [13]:
socialGraph.getVertexTypes()

['person']

In [14]:
socialGraph.getEdgeStats('friendship', skipNA=False)

{'friendship': {'connect_day': {'MAX': 1496448000,
   'MIN': 1420070400,
   'AVG': 1458864000}}}

In [15]:
socialGraph.getEdges('person', 'Jenny'
              , edgeType='friendship'
              , targetVertexType='person'
              , targetVertexId=None, select="connect_day", where="", limit="", sort="", timeout=0)

[{'e_type': 'friendship',
  'directed': False,
  'from_id': 'Jenny',
  'from_type': 'person',
  'to_id': 'Amily',
  'to_type': 'person',
  'attributes': {'connect_day': '2015-06-08 00:00:00'}},
 {'e_type': 'friendship',
  'directed': False,
  'from_id': 'Jenny',
  'from_type': 'person',
  'to_id': 'Tom',
  'to_type': 'person',
  'attributes': {'connect_day': '2015-01-01 00:00:00'}},
 {'e_type': 'friendship',
  'directed': False,
  'from_id': 'Jenny',
  'from_type': 'person',
  'to_id': 'Dan',
  'to_type': 'person',
  'attributes': {'connect_day': '2016-08-03 00:00:00'}}]

In [16]:
#getEdgesDataframe(sourceVertexType, sourceVerticies, edgeType=None, targetVertexType=None, targetVertexId=None, select="", where="", limit="", sort="", timeout=0)

df=socialGraph.getEdgesDataframe("person", "Dan")
df

,from_type,from_id,to_type,to_id,connect_day
0,person,Dan,person,Tom,2017-06-03 00:00:00
1,person,Dan,person,Kevin,2015-12-30 00:00:00
2,person,Dan,person,Jenny,2016-08-03 00:00:00
3,person,Dan,person,Nancy,2016-01-03 00:00:00


In [17]:
resultSet1 = socialGraph.gsql('''use graph social 
   SELECT * FROM person LIMIT 3''')
resultSet1

'Using graph \'social\'\n[\n{\n"v_id": "Nancy",\n"attributes": {\n"gender": "female",\n"name": "Nancy",\n"state": "ky",\n"age": 20\n},\n"v_type": "person"\n},\n{\n"v_id": "Jenny",\n"attributes": {\n"gender": "female",\n"name": "Jenny",\n"state": "tx",\n"age": 25\n},\n"v_type": "person"\n},\n{\n"v_id": "Jack",\n"attributes": {\n"gender": "male",\n"name": "Jack",\n"state": "fl",\n"age": 26\n},\n"v_type": "person"\n}\n]'

In [18]:
results = socialGraph.gsql('''SELECT * from person:p-(friendship:f)-person:t LIMIT 5''')
results

'Encountered " ":" ": "" at line 1, column 21.\nWas expecting one of:\n<EOF>\n"limit" ...\n"order" ...\n"where" ...\n"-(" ...\n'

In [19]:
results = socialGraph.gsql('''use graph social
                        select * from person where gender=="female"''')
results
#df = pd.DataFrame(results[0]['attributes'])
#df
#print(json.dumps(results, indent=2))

'Using graph \'social\'\n[\n{\n"v_id": "Nancy",\n"attributes": {\n"gender": "female",\n"name": "Nancy",\n"state": "ky",\n"age": 20\n},\n"v_type": "person"\n},\n{\n"v_id": "Jenny",\n"attributes": {\n"gender": "female",\n"name": "Jenny",\n"state": "tx",\n"age": 25\n},\n"v_type": "person"\n},\n{\n"v_id": "Amily",\n"attributes": {\n"gender": "female",\n"name": "Amily",\n"state": "ca",\n"age": 22\n},\n"v_type": "person"\n}\n]'

In [20]:
res=socialGraph.getVertices('person', select='name,age,gender', where='gender=="female"')
res

[{'v_id': 'Nancy',
  'v_type': 'person',
  'attributes': {'name': 'Nancy', 'age': 20, 'gender': 'female'}},
 {'v_id': 'Jenny',
  'v_type': 'person',
  'attributes': {'name': 'Jenny', 'age': 25, 'gender': 'female'}},
 {'v_id': 'Amily',
  'v_type': 'person',
  'attributes': {'name': 'Amily', 'age': 22, 'gender': 'female'}}]

In [21]:
type(res)

list

In [22]:
attrs=[x['attributes'] for x in res]
attrs

[{'name': 'Nancy', 'age': 20, 'gender': 'female'},
 {'name': 'Jenny', 'age': 25, 'gender': 'female'},
 {'name': 'Amily', 'age': 22, 'gender': 'female'}]

In [23]:
type(attrs)

list

In [24]:
attrs=res[-2]['attributes']
attrs

{'name': 'Jenny', 'age': 25, 'gender': 'female'}

In [25]:
df = pd.DataFrame(res)
df

,v_id,v_type,attributes
0,Nancy,person,"{'name': 'Nancy', 'age': 20, 'gender': 'female'}"
1,Jenny,person,"{'name': 'Jenny', 'age': 25, 'gender': 'female'}"
2,Amily,person,"{'name': 'Amily', 'age': 22, 'gender': 'female'}"


In [26]:
sourceVertexType='person'
sourceVertexId='Dan'
socialGraph.getEdges(sourceVertexType, sourceVertexId, edgeType=None, targetVertexType=None, targetVertexId=None, select="", where="", limit="", sort="", timeout=0)

[{'e_type': 'friendship',
  'directed': False,
  'from_id': 'Dan',
  'from_type': 'person',
  'to_id': 'Tom',
  'to_type': 'person',
  'attributes': {'connect_day': '2017-06-03 00:00:00'}},
 {'e_type': 'friendship',
  'directed': False,
  'from_id': 'Dan',
  'from_type': 'person',
  'to_id': 'Kevin',
  'to_type': 'person',
  'attributes': {'connect_day': '2015-12-30 00:00:00'}},
 {'e_type': 'friendship',
  'directed': False,
  'from_id': 'Dan',
  'from_type': 'person',
  'to_id': 'Jenny',
  'to_type': 'person',
  'attributes': {'connect_day': '2016-08-03 00:00:00'}},
 {'e_type': 'friendship',
  'directed': False,
  'from_id': 'Dan',
  'from_type': 'person',
  'to_id': 'Nancy',
  'to_type': 'person',
  'attributes': {'connect_day': '2016-01-03 00:00:00'}}]

## Step 5: Write Queries

<div>
  <img style="vertical-align:top" src="img/query.png" width="28" height="28"/>
  <span style="">TBD. 
</span>
</div>

In [27]:
socialGraph.gsql('''use graph social
                 select * from person where primary_id=="Tom"''')

'Using graph \'social\'\n[{\n"v_id": "Tom",\n"attributes": {\n"gender": "male",\n"name": "Tom",\n"state": "ca",\n"age": 40\n},\n"v_type": "person"\n}]'

In [28]:
socialGraph.runInterpretedQuery('''
  INTERPRET QUERY x() FOR GRAPH social {
  # declaration statements
  STRING uid = "Tom";
  users = {person.*};
  # body statements
  posts = SELECT p
    FROM users:u-(friendship)->:p
    WHERE u.name == uid;
  PRINT posts; 
}
''')

[{'posts': [{'v_id': 'Dan',
    'v_type': 'person',
    'attributes': {'gender': 'male', 'name': 'Dan', 'age': 34, 'state': 'ny'}},
   {'v_id': 'Jenny',
    'v_type': 'person',
    'attributes': {'gender': 'female',
     'name': 'Jenny',
     'age': 25,
     'state': 'tx'}}]}]

In [29]:
socialGraph.runInterpretedQuery('''
  INTERPRET QUERY () FOR GRAPH social {
    PRINT "Hello World"; 
}
''')

[{'"Hello World"': 'Hello World'}]

In [30]:
socialGraph.runInterpretedQuery('''
  INTERPRET QUERY () FOR GRAPH social {
    person1 = {person.*};
    Result = SELECT tgt
           FROM person1:s-(friendship:e)-person:tgt;
    PRINT Result; 
}
''')

[{'Result': [{'v_id': 'Amily',
    'v_type': 'person',
    'attributes': {'gender': 'female',
     'name': 'Amily',
     'age': 22,
     'state': 'ca'}},
   {'v_id': 'Tom',
    'v_type': 'person',
    'attributes': {'gender': 'male', 'name': 'Tom', 'age': 40, 'state': 'ca'}},
   {'v_id': 'Kevin',
    'v_type': 'person',
    'attributes': {'gender': 'male',
     'name': 'Kevin',
     'age': 28,
     'state': 'az'}},
   {'v_id': 'Jack',
    'v_type': 'person',
    'attributes': {'gender': 'male',
     'name': 'Jack',
     'age': 26,
     'state': 'fl'}},
   {'v_id': 'Nancy',
    'v_type': 'person',
    'attributes': {'gender': 'female',
     'name': 'Nancy',
     'age': 20,
     'state': 'ky'}},
   {'v_id': 'Jenny',
    'v_type': 'person',
    'attributes': {'gender': 'female',
     'name': 'Jenny',
     'age': 25,
     'state': 'tx'}},
   {'v_id': 'Dan',
    'v_type': 'person',
    'attributes': {'gender': 'male',
     'name': 'Dan',
     'age': 34,
     'state': 'ny'}}]}]